# Assignment 2 - SU22

### Group 19 - Ruchi Sharma, Soumith Palreddy, Disha Gandhi

In [1]:
import pandas as pd

In [11]:
df = pd.read_csv('NYC_Restaurants.csv', dtype=str)

#### Question 10

In [23]:
df['RESTAURANT'] = df['DBA'] + " " + df['BUILDING'] + " " + df['STREET'] + " " + df['ZIPCODE'] + " " + df['BORO']
#Select all cuisines for which there were at least 20 restaurants representing that cuisine.
temp = df[['CUISINE DESCRIPTION', 'RESTAURANT']]['CUISINE DESCRIPTION'].value_counts() >= 20
cuisines_with_greater_than_20_restaurants = pd.DataFrame(temp).reset_index().rename(columns = {'index': 'NAME'})
#converting the boolean data as obtained above into dataframe and obtaining other columns
cuisines_with_greater_than_20_restaurants = cuisines_with_greater_than_20_restaurants[cuisines_with_greater_than_20_restaurants['CUISINE DESCRIPTION'] == True]['NAME'] # Series with atlest 20
cuisines_with_greater_than_20_restaurants = cuisines_with_greater_than_20_restaurants.to_frame()

In [24]:
#same function as borrowed from question 9 above
ques9 = df[df['VIOLATION CODE'].isna()].drop_duplicates(subset = ['RESTAURANT', 'CUISINE DESCRIPTION'])[
                                            'CUISINE DESCRIPTION'].value_counts().reset_index().rename(
                                                                            columns = {'index': 'NAME'})

#same function as borrowed from question 8 above
ques8 = df.drop_duplicates(subset = ['RESTAURANT', 'CUISINE DESCRIPTION'])[
                                            'CUISINE DESCRIPTION'].value_counts().reset_index().rename(
                                                                            columns = {'index': 'NAME'})
#once cusines are selected that satisfy the condition we perform two merges to get both the ratios from questions 9 and 8
#in the same dataframe. We perform the merge based on the uniqueness of name
all_ratios_for_cuisine = cuisines_with_greater_than_20_restaurants.merge(ques9, left_on = 'NAME', right_on = 'NAME').merge(
                                                        ques8, left_on = 'NAME', right_on = 'NAME')
#calculating the final ratio
all_ratios_for_cuisine['FINAL RATIO OF NON CITED RESTAURANTS'] = all_ratios_for_cuisine['CUISINE DESCRIPTION_x']/all_ratios_for_cuisine['CUISINE DESCRIPTION_y']
all_ratios_for_cuisine.sort_values(by = 'FINAL RATIO OF NON CITED RESTAURANTS', ascending = False)[:10]

,NAME,CUISINE DESCRIPTION_x,CUISINE DESCRIPTION_y,FINAL RATIO OF NON CITED RESTAURANTS
53,Creole,2,11,0.181818
51,Chinese/Cuban,2,11,0.181818
39,Bangladeshi,3,20,0.150000
37,Russian,4,36,0.111111
38,Peruvian,3,28,0.107143
50,Soups & Sandwiches,2,21,0.095238
49,Continental,2,21,0.095238
45,Barbecue,2,21,0.095238
34,Vietnamese/Cambodian/Malaysia,3,32,0.093750
44,Soul Food,2,22,0.090909


#### Question 11

In [6]:
#dropping all those rows having Missing as a value for borough
df = df.drop(df[df.BORO == 'Missing'].index)
#applying crosstab to calculate different violation descriptions across different borough
violation_description_df = pd.crosstab(df['BORO'], df['VIOLATION DESCRIPTION'], rownames=['BORO'], colnames=['VIOLATION DESCRIPTION'])
display(violation_description_df)
#finding out the violation description most prevalant in each borough
maxValueIndex = violation_description_df.idxmax(axis = 1)
 
print("Max values of row are at following columns :")
print(maxValueIndex)

VIOLATION DESCRIPTION,"Choking first aid poster not posted. Alcohol and pregnancy warning sign not posted. Resuscitation equipment: exhaled air resuscitation masks (adult & pediatric), latex gloves, sign not posted. Inspection report sign not posted.",''''No Smoking and/or 'Smoking Permitted sign not conspicuously posted. Health warning not present on 'Smoking Permitted,''''Wash hands sign not posted at hand wash facility.,"A food containing artificial trans fat, with 0.5 grams or more of trans fat per serving, is being stored, distributed, held for service, used in preparation of a menu item, or served.",Accurate thermometer not provided in refrigerated or hot holding equipment.,"Appropriately scaled metal stem-type thermometer or thermocouple not provided or used to evaluate temperatures of potentially hazardous foods during cooking, cooling, reheating and holding.",Ashtray present in smoke-free area.,"Bulb not shielded or shatterproof, in areas where there is extreme heat, temperature changes, or where accidental contact may occur.","Caloric content not posted on menus, menu boards or food tags, in a food service establishment that is 1 of 15 or more outlets operating the same type of business nationally under common ownership or control, or as a franchise or doing business under the same name, for each menu item that is served in portions, the size and content of which are standardized.","Caloric content range (minimum to maximum) not posted on menus and or menu boards for each flavor, variety and size of each menu item that is offered for sale in different flavors, varieties and sizes.",...,"Smoke free workplace smoking policy inadequate, not posted, not provided to employees.",Thawing procedures improper.,"The original nutritional fact labels and/or ingredient label for a cooking oil, shortening or margarine or food item sold in bulk, or acceptable manufacturers documentation not maintained on site.","Tobacco use, eating, or drinking from open container in food preparation, food storage or dishwashing area observed.","Toilet facility not maintained and provided with toilet paper, waste receptacle and self-closing door.",Toilet facility not provided for employees or for patrons when required.,Toilet facility used by women does not have at least one covered garbage receptacle.,"Toxic chemical improperly labeled, stored or used such that food contamination may occur.",Unprotected food re-served.,Wiping cloths soiled or not stored in sanitizing solution.
BORO,,,,,,,,,,,,,,,,,,,,,
BRONX,12,1,21,5,21,27,4,20,0,0,...,45,36,48,22,46,2,1,2,1,47
BROOKLYN,40,5,57,12,76,120,4,93,4,0,...,222,109,125,55,142,3,5,8,0,198
MANHATTAN,49,6,93,19,98,162,5,168,6,0,...,264,151,198,107,225,11,7,15,1,285
QUEENS,29,4,60,24,92,85,8,80,5,1,...,154,100,151,64,134,3,2,20,0,168
STATEN ISLAND,5,1,8,3,16,17,1,10,1,0,...,22,23,25,7,14,1,1,3,0,25


Max values of row are at following columns :
BORO
BRONX            Non-food contact surface improperly constructe...
BROOKLYN         Non-food contact surface improperly constructe...
MANHATTAN        Non-food contact surface improperly constructe...
QUEENS           Non-food contact surface improperly constructe...
STATEN ISLAND    Non-food contact surface improperly constructe...
dtype: object


#### Question 12

In [7]:
#making a new row containing the total count of violation description across all boroughs
violation_description_df.loc['violationFrequency'] = violation_description_df.sum(numeric_only=True, axis=0)
#normalizing count across all violations in each borough
violation_description_df.loc[:,'\x1aChoking first aid\x1a poster not posted. \x1aAlcohol and pregnancy\x1a warning sign not posted. Resuscitation equipment: exhaled air resuscitation masks (adult & pediatric), latex gloves, sign not posted. Inspection report sign not posted.':] = violation_description_df.loc[:,'\x1aChoking first aid\x1a poster not posted. \x1aAlcohol and pregnancy\x1a warning sign not posted. Resuscitation equipment: exhaled air resuscitation masks (adult & pediatric), latex gloves, sign not posted. Inspection report sign not posted.':]/violation_description_df.iloc[-1]['\x1aChoking first aid\x1a poster not posted. \x1aAlcohol and pregnancy\x1a warning sign not posted. Resuscitation equipment: exhaled air resuscitation masks (adult & pediatric), latex gloves, sign not posted. Inspection report sign not posted.':]
display(violation_description_df)
#finding out the violation description most prevalant in each borough after normalization
maxValueIndex = violation_description_df.idxmax(axis = 1)
 
print("Max values of row are at following columns :")
print(maxValueIndex)

VIOLATION DESCRIPTION,"Choking first aid poster not posted. Alcohol and pregnancy warning sign not posted. Resuscitation equipment: exhaled air resuscitation masks (adult & pediatric), latex gloves, sign not posted. Inspection report sign not posted.",''''No Smoking and/or 'Smoking Permitted sign not conspicuously posted. Health warning not present on 'Smoking Permitted,''''Wash hands sign not posted at hand wash facility.,"A food containing artificial trans fat, with 0.5 grams or more of trans fat per serving, is being stored, distributed, held for service, used in preparation of a menu item, or served.",Accurate thermometer not provided in refrigerated or hot holding equipment.,"Appropriately scaled metal stem-type thermometer or thermocouple not provided or used to evaluate temperatures of potentially hazardous foods during cooking, cooling, reheating and holding.",Ashtray present in smoke-free area.,"Bulb not shielded or shatterproof, in areas where there is extreme heat, temperature changes, or where accidental contact may occur.","Caloric content not posted on menus, menu boards or food tags, in a food service establishment that is 1 of 15 or more outlets operating the same type of business nationally under common ownership or control, or as a franchise or doing business under the same name, for each menu item that is served in portions, the size and content of which are standardized.","Caloric content range (minimum to maximum) not posted on menus and or menu boards for each flavor, variety and size of each menu item that is offered for sale in different flavors, varieties and sizes.",...,"Smoke free workplace smoking policy inadequate, not posted, not provided to employees.",Thawing procedures improper.,"The original nutritional fact labels and/or ingredient label for a cooking oil, shortening or margarine or food item sold in bulk, or acceptable manufacturers documentation not maintained on site.","Tobacco use, eating, or drinking from open container in food preparation, food storage or dishwashing area observed.","Toilet facility not maintained and provided with toilet paper, waste receptacle and self-closing door.",Toilet facility not provided for employees or for patrons when required.,Toilet facility used by women does not have at least one covered garbage receptacle.,"Toxic chemical improperly labeled, stored or used such that food contamination may occur.",Unprotected food re-served.,Wiping cloths soiled or not stored in sanitizing solution.
BORO,,,,,,,,,,,,,,,,,,,,,
BRONX,0.088889,0.058824,0.087866,0.079365,0.069307,0.065693,0.181818,0.053908,0.0000,0.0,...,0.063649,0.085919,0.087751,0.086275,0.081996,0.10,0.0625,0.041667,0.5,0.065007
BROOKLYN,0.296296,0.294118,0.238494,0.190476,0.250825,0.291971,0.181818,0.250674,0.2500,0.0,...,0.314003,0.260143,0.228519,0.215686,0.253119,0.15,0.3125,0.166667,0.0,0.273859
MANHATTAN,0.362963,0.352941,0.389121,0.301587,0.323432,0.394161,0.227273,0.452830,0.3750,0.0,...,0.373409,0.360382,0.361974,0.419608,0.401070,0.55,0.4375,0.312500,0.5,0.394191
QUEENS,0.214815,0.235294,0.251046,0.380952,0.303630,0.206813,0.363636,0.215633,0.3125,1.0,...,0.217822,0.238663,0.276051,0.250980,0.238859,0.15,0.1250,0.416667,0.0,0.232365
STATEN ISLAND,0.037037,0.058824,0.033473,0.047619,0.052805,0.041363,0.045455,0.026954,0.0625,0.0,...,0.031117,0.054893,0.045704,0.027451,0.024955,0.05,0.0625,0.062500,0.0,0.034578
violationFrequency,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0000,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.00,1.0000,1.000000,1.0,1.000000


Max values of row are at following columns :
BORO
BRONX                                       Unprotected food re-served.
BROOKLYN              Precooked potentially hazardous food from comm...
MANHATTAN             Out-of package sale of tobacco products observed.
QUEENS                Caloric content range (minimum to maximum) not...
STATEN ISLAND         Eggs found dirty/cracked; liquid, frozen or po...
violationFrequency    Choking first aid poster not posted. Alcoho...
dtype: object


#### Question 13

In [12]:
#Generating areacode with first three charachters of Phone
df['AREACODE'] = df.PHONE.apply(lambda x: x[:3])
#Converting Phone number to string
df['PHONE'] = df['PHONE'].astype(str)
#Grouping by on area code and zip code and then counting the combinations where one area code corresponds to only one zip code
non_unique = df.groupby('AREACODE').ZIPCODE.transform('nunique').ne(1)
ndf = df[~non_unique]
#taking the length of such combinations
len(ndf.groupby(['AREACODE','ZIPCODE']).size().reset_index().rename(columns={0:'count'}))

34

#### Question 14

In [13]:
#Create a new column, called STREET TYPE, which is the the last word in the name of the street. For example, if the street is “Astoria Boule- vard”, the street type should be “Boulevard”.
df['STREET TYPE'] = df.STREET.astype(str).str.split().str[-1]
#Create another column, called STREET BASE, which contains everything but the last word in the name of the street. For example, if the street is “Astoria Boulevard”, the street base should be “Astoria”.
df['STREET BASE'] = df.STREET.str.rsplit(' ',1).str[0]
#Create a third column, called STREET BASE & ZIP, that combines the street base and the zipcode.
df['STREET BASE & ZIP'] = df['STREET BASE'] + ' ' + df['ZIPCODE'].astype(str)
#Create a table containing just these three columns, and remove any duplicates or instances where street base is empty. This table now contains unique street names, along with the street type.
new_df = df[['STREET TYPE', 'STREET BASE', 'STREET BASE & ZIP']].copy()
new_df = new_df.drop_duplicates();
new_df = new_df.dropna(subset=['STREET BASE'])
#Merge this table with itself, on the STREET BASE & ZIP column. Thus, in the new merged tabled, we will have two STREET TYPE fields for each street base and zipcode.
new_merged_df = pd.merge(new_df,new_df,on=["STREET BASE & ZIP"])
#From the merged table, select only the rows where the street types are different.
#mismatch_streets_df will have all the pairs that are mismatched with street type x and street type y columns being compared.
mismatch_streets_df = pd.DataFrame()
for index, row in new_merged_df.iterrows():
    if row['STREET TYPE_x'] != row['STREET TYPE_y']:
        mismatch_streets_df = mismatch_streets_df.append(row, ignore_index=True)

#Now, do a cross-tabulation of the two distinct street types (check out the crosstab function in Pandas). This gives us the number of times Ave and Avenue were used with the same STREET BASE & ZIP.
freq_of_street_types = pd.crosstab(mismatch_streets_df['STREET BASE & ZIP'], [mismatch_streets_df['STREET TYPE_x'], mismatch_streets_df['STREET TYPE_y']], rownames=['STREET BASE & ZIP'], colnames=['STREET TYPE_x', 'STREET TYPE_y'])
display()
#making a new row containing the total count of street type across all street base and zip's
freq_of_street_types.loc['total'] = freq_of_street_types.sum(numeric_only=True, axis=0)
#list of street types given in the question
list_of_street_types_to_match=['AVE','ST','RD','PL','BOULEARD','BULEVARD']
#for this list we check which street type is present in the crosstab columns and find the maximum in the total row,
#wherever the max is found we put that column pair in the final list of mismatched street type pairs
list_of_mismatch_pairs =[]
for element in list_of_street_types_to_match:
    all_columns_list_for_street_type = [num for num in freq_of_street_types.columns.tolist() if element in num]
    temp_df = freq_of_street_types[all_columns_list_for_street_type].copy()
    seleted_mismatch_pair_tuple = temp_df.iloc[-1].idxmax(axis=1)
    list_of_mismatch_pairs.append(seleted_mismatch_pair_tuple)

list_of_mismatch_pairs

[('AVE', 'AVENUE'),
 ('ST', 'STREET'),
 ('RD', 'ROAD'),
 ('PL', 'PLACE'),
 ('BLVD', 'BOULEARD'),
 ('BLVD', 'BULEVARD')]